In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lupus_(Systemic_Lupus_Erythematosus)"
cohort = "GSE112943"

# Input paths
in_trait_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)"
in_cohort_dir = "../../input/GEO/Lupus_(Systemic_Lupus_Erythematosus)/GSE112943"

# Output paths
out_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE112943.csv"
out_gene_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE112943.csv"
out_clinical_data_file = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/clinical_data/GSE112943.csv"
json_path = "../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide analysis of gene expression of cutaneous lupus skin samples and lupus nephritis kidney samples"
!Series_summary	"Microarray gene expression analyses were performed on human skin samples from cutaneous lupus subtypes (SCLE and CCLE) and normal patients along with human kidney samples from lupus nephritis and normal patients"
!Series_overall_design	"47 deidentified human samples from formalin fixed, paraffin-embedded skin (6 chronic cutaneous lupus, 10 subacute cutaneous lupus, 10 control skin) and formalin fixed paraffin-embedded kidney (14 lupus nephritis, 7 control kidney)"
Sample Characteristics Dictionary:
{0: ['tissue: CCLE', 'tissue: SCLE', 'tissue: Skin Control', 'tissue: Kidney Control', 'tissue: Lupus Nephritis']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarray analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary, row 0 contains tissue information
# This can be used to determine lupus status (trait)
trait_row = 0
# Age and gender information are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value_str):
    """Convert tissue information to binary trait (Lupus vs Control)"""
    if not isinstance(value_str, str):
        return None
    
    if ':' in value_str:
        value = value_str.split(':', 1)[1].strip()
    else:
        value = value_str.strip()
    
    # CCLE = Chronic Cutaneous Lupus Erythematosus
    # SCLE = Subacute Cutaneous Lupus Erythematosus
    # Lupus Nephritis is also a form of lupus
    if 'CCLE' in value or 'SCLE' in value or 'Lupus Nephritis' in value:
        return 1  # Lupus
    elif 'Control' in value:
        return 0  # Control
    else:
        return None

def convert_age(value_str):
    """Convert age string to numeric value (not used in this dataset)"""
    return None  # Age data not available

def convert_gender(value_str):
    """Convert gender string to binary (not used in this dataset)"""
    return None  # Gender data not available

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Use the function to extract clinical features
    try:
        # Assuming clinical_data is already available from previous step
        # If clinical_data is not available or loaded, we need to handle it
        if 'clinical_data' in locals() or 'clinical_data' in globals():
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the extracted clinical features
            preview = preview_df(selected_clinical_df)
            print("Clinical Data Preview:")
            print(preview)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the clinical data to CSV
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("Clinical data not available from previous step")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")


Clinical Data Preview:
{'GSM3091745': [1.0], 'GSM3091746': [1.0], 'GSM3091747': [1.0], 'GSM3091748': [1.0], 'GSM3091749': [1.0], 'GSM3091750': [1.0], 'GSM3091751': [1.0], 'GSM3091752': [1.0], 'GSM3091753': [1.0], 'GSM3091754': [1.0], 'GSM3091755': [1.0], 'GSM3091756': [1.0], 'GSM3091757': [1.0], 'GSM3091758': [1.0], 'GSM3091759': [1.0], 'GSM3091760': [1.0], 'GSM3091765': [0.0], 'GSM3091766': [0.0], 'GSM3091767': [0.0], 'GSM3091768': [0.0], 'GSM3091769': [0.0], 'GSM3091770': [0.0], 'GSM3091771': [0.0], 'GSM3091772': [0.0], 'GSM3091773': [0.0], 'GSM3091774': [0.0], 'GSM3091775': [0.0], 'GSM3091776': [0.0], 'GSM3091777': [0.0], 'GSM3091778': [0.0], 'GSM3091779': [0.0], 'GSM3091780': [0.0], 'GSM3091781': [0.0], 'GSM3091782': [1.0], 'GSM3091783': [1.0], 'GSM3091784': [1.0], 'GSM3091785': [1.0], 'GSM3091786': [1.0], 'GSM3091787': [1.0], 'GSM3091788': [1.0], 'GSM3091789': [1.0], 'GSM3091790': [1.0], 'GSM3091791': [1.0], 'GSM3091792': [1.0], 'GSM3091793': [1.0], 'GSM3091794': [1.0], 'GSM309179

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 47303
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review of gene identifiers in the gene expression data
# The identifiers start with "ILMN_" which indicates they're Illumina BeadChip probe IDs
# These are microarray probe identifiers, not human gene symbols
# They need to be mapped to official gene symbols for better interpretability and cross-platform analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. After observing the gene annotation data and gene expression data:
# - The gene identifiers in gene expression data start with "ILMN_"
# - In the annotation data, these are stored in the 'ID' column
# - The gene symbols are stored in the 'Symbol' column

# 2. Create a gene mapping dataframe with the probe ID and gene symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Let's examine the mapping dataframe
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize gene symbols (handle synonyms)
gene_data = normalize_gene_symbols_in_index(gene_data)

# Preview the converted gene expression data
print("\nGene-level expression data preview:")
print(f"Shape: {gene_data.shape}")
print(f"Number of genes: {len(gene_data)}")
print(f"First few gene symbols: {list(gene_data.index[:10])}")

# Save the gene data to the specified output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}



Gene-level expression data preview:
Shape: (20254, 47)
Number of genes: 20254
First few gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Gene data saved to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE112943.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Load the clinical data 
# Load the clinical data that was already processed in step 2
selected_clinical_df = pd.read_csv(out_clinical_data_file)
print(f"Loaded clinical data with shape: {selected_clinical_df.shape}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# Inspect the column names to find the trait column name
print(f"Column names in linked data: {linked_data.columns[:5].tolist()}")

# 4. Handle missing values in the linked data
# Since we're dealing with a trait that was saved from a previous step,
# we need to find the actual column name used for the trait in the linked data
# The first column (index 0) of clinical data should be our trait
trait_col = linked_data.columns[0]
print(f"Using '{trait_col}' as the trait column for handling missing values")
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 5. Determine if the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait_col)

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from skin and kidney samples of lupus patients and controls."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Final linked data not saved.")

Top 10 gene indices before normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']
Top 10 gene indices after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']
Shape of normalized gene data: (20254, 47)


Saved normalized gene data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/gene_data/GSE112943.csv
Loaded clinical data with shape: (1, 47)
Shape of linked data: (47, 20255)
Column names in linked data: [0, 'A1BG', 'A1BG-AS1', 'A1CF', 'A2M']
Using '0' as the trait column for handling missing values


Shape of linked data after handling missing values: (47, 20255)
For the feature '0', the least common label is '0.0' with 17 occurrences. This represents 36.17% of the dataset.
The distribution of the feature '0' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/cohort_info.json


Saved processed linked data to ../../output/preprocess/Lupus_(Systemic_Lupus_Erythematosus)/GSE112943.csv
